In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from vacances_scolaires_france import SchoolHolidayDates
import datetime
pd.set_option('display.max_columns', 500)



# df = pd.read_csv('../films_db2.csv')
# df.head(10)

In [2]:
from db.database_mysql import engine

df = pd.read_sql_query('''SELECT jp.raw_title, 
        YEAR(jp.date) AS year, 
        MONTH(jp.date) AS month, 
        DAY(jp.date) AS day, 
    im.director, im.date,im.casting, im.distributor, jp.country, jp.duration, jp.genre, jp.first_day, jp.first_week, jp.first_weekend, jp.hebdo_rank, 
jp.total_spectator, jp.copies, im.rating_press, im.budget, im.lang, im.award 
FROM films_jp as jp
LEFT JOIN films_imdb im ON im.id_jp = jp.id 
where im.id_jp is not null and im.date = jp.date
order by jp.first_week desc''', engine)

df

,raw_title,year,month,day,director,date,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[""kad merad"", ""dany boon"", ""zoe felix""]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[""josiane balasko"", ""michel blanc"", ""marieanne...","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[""samy naceri"", ""frederic diefenthal"", ""emma w...","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[""hayden christensen"", ""natalie portman"", ""ewa...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[""donald glover"", ""beyonce"", ""seth rogen""]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""",2010-09-22,"[""jules pelissier"", ""ana girardot"", ""audrey ba...","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351,48033,40282,15,79945,93,6.4,-1,"[""francais""]",3
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""",2003-12-10,"[""michael pitt"", ""louis garrel"", ""eva green""]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577,47922,-1,13,79849,-1,7.1,15000000,"[""anglais"", ""francais""]",2
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[""toni servillo"", ""carlo verdone"", ""sabrina fe...","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654,47900,36844,13,198373,81,7.7,9200000,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60
2668,Un baiser papillon,2011,6,1,"""karine silla""",2011-06-01,"[""valeria golino"", ""elsa zylberstein"", ""vincen...","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,-1,47715,38189,14,47715,170,5.4,3500000,"[""francais""]",2


In [3]:
df.isnull().sum()

raw_title          0
year               0
month              0
day                0
director           1
date               0
casting            0
distributor        0
country            0
duration           0
genre              0
first_day          0
first_week         0
first_weekend      0
hebdo_rank         0
total_spectator    0
copies             0
rating_press       0
budget             0
lang               0
award              0
dtype: int64

In [4]:
# df = df.dropna()

In [5]:
df.nunique()

raw_title          2668
year                 32
month                12
day                  31
director           1501
date               1355
casting            2637
distributor        2469
country              35
duration            115
genre                18
first_day          2207
first_week         2659
first_weekend      2082
hebdo_rank           20
total_spectator    2664
copies              742
rating_press         58
budget              391
lang                551
award               107
dtype: int64

In [6]:
# realisateurs_counts = df['director'].value_counts()

# realisateurs_counts_df = pd.DataFrame(realisateurs_counts)
# realisateurs_counts_df.reset_index(inplace=True)
# realisateurs_counts_df.columns = ['director', 'Nombre de films']

In [7]:
# spectateurs_par_realisateur = df.groupby('director')['total_spectator'].sum().reset_index()

In [8]:
# awards_par_realisateur = df.groupby('director')['award'].sum().reset_index()

In [9]:
# merged_df = pd.merge(realisateurs_counts_df, spectateurs_par_realisateur, left_on='director', right_on='director', how='left')

# # merged_df['director_score'] = df['first_week'] / realisateurs_counts_df['Nombre de films'] * (awards_par_realisateur['award'] +1)
# merged_df['director_score'] = df['first_week'] / realisateurs_counts_df['Nombre de films']



In [10]:
# df = pd.merge(df, merged_df[['director', 'director_score']], on='director', how='left')

In [11]:
def calculate_director_scores(df):
    # 1. Fréquence de réalisation
    frequency = df['director'].value_counts()

    # 2. Moyenne des spectateurs totaux
    avg_total_spectators = df.groupby('director')['total_spectator'].mean()

    # 3. Moyenne de la première semaine
    avg_first_week = df.groupby('director')['first_week'].mean()

    # 4. Moyenne inversée du classement hebdomadaire
    avg_inv_hebdo_rank = 1 / df.groupby('director')['hebdo_rank'].mean()

    # 5. Moyenne des évaluations de la presse
    avg_rating_press = df.groupby('director')['rating_press'].mean()

    # 6. Rendement (total_spectator / budget)
    performance = (df.groupby('director')['total_spectator'].sum() / df.groupby('director')['budget'].sum())

    # Compilation des scores dans un DataFrame
    director_scores = pd.DataFrame({
        'frequency': frequency,
        'avg_total_spectators': avg_total_spectators,
        'avg_first_week': avg_first_week,
        'avg_inv_hebdo_rank': avg_inv_hebdo_rank,
        'avg_rating_press': avg_rating_press,
        'performance': performance
    })

    # Normalisation des scores
    max_values = director_scores.max()
    normalized_scores = director_scores / max_values

    # Combinaison des scores normalisés
    normalized_scores['director_combined_score'] = normalized_scores.mean(axis=1)

    return normalized_scores

# Appliquer la fonction pour calculer les scores
director_scores = calculate_director_scores(df)

# Joindre les scores combinés avec le DataFrame original
df = df.join(director_scores['director_combined_score'], on='director')
df 


,raw_title,year,month,day,director,date,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[""kad merad"", ""dany boon"", ""zoe felix""]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5,0.537844
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[""josiane balasko"", ""michel blanc"", ""marieanne...","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1,0.288597
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[""samy naceri"", ""frederic diefenthal"", ""emma w...","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1,0.447876
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[""hayden christensen"", ""natalie portman"", ""ewa...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29,0.564055
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[""donald glover"", ""beyonce"", ""seth rogen""]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21,0.348963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""",2010-09-22,"[""jules pelissier"", ""ana girardot"", ""audrey ba...","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351,48033,40282,15,79945,93,6.4,-1,"[""francais""]",3,0.131239
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""",2003-12-10,"[""michael pitt"", ""louis garrel"", ""eva green""]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577,47922,-1,13,79849,-1,7.1,15000000,"[""anglais"", ""francais""]",2,0.167568
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[""toni servillo"", ""carlo verdone"", ""sabrina fe...","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654,47900,36844,13,198373,81,7.7,9200000,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60,0.181678
2668,Un baiser papillon,2011,6,1,"""karine silla""",2011-06-01,"[""valeria golino"", ""elsa zylberstein"", ""vincen...","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,-1,47715,38189,14,47715,170,5.4,3500000,"[""francais""]",2,0.133003


In [12]:
df['entree_annee'] = df['year']


def classify_entrees_year(entrees):
    if entrees == 1986:
        return 168.1
    elif entrees == 1992:
        return 116.0
    elif entrees == 1993:
        return 132.7
    elif entrees == 1994:
        return 124.4
    elif entrees == 1995:
        return 130.2
    elif entrees == 1996:
        return 136.7
    elif entrees == 1997:
        return 149.3
    elif entrees == 1998:
        return 170.6
    elif entrees == 1999:
        return 153.6
    elif entrees == 2000:
        return 165.8
    elif entrees == 2001:
        return 187.5
    elif entrees == 2002:
        return 184.4
    elif entrees == 2003:
        return 173.5
    elif entrees == 2004:
        return 195.8
    elif entrees == 2005:
        return 175.6
    elif entrees == 2006:
        return 188.8
    elif entrees == 2007:
        return 178.5
    elif entrees == 2008:
        return 190.3
    elif entrees == 2009:
        return 201.6
    elif entrees == 2010:
        return 207.1
    elif entrees == 2011:
        return 217.2
    elif entrees == 2012:
        return 203.6
    elif entrees == 2013:
        return 193.7
    elif entrees == 2014:
        return 209.1
    elif entrees == 2015:
        return 205.4
    elif entrees == 2016:
        return 213.2
    elif entrees == 2017:
        return 209.4
    elif entrees == 2018:
        return 201.2
    elif entrees == 2019:
        return 213.2
    elif entrees == 2020:
        return 65.3
    elif entrees == 2021:
        return 95.5
    elif entrees == 2022:
        return 	152.0
    elif entrees == 2023 or 2024:
        return 180.8



df['entree_annee'] = df['entree_annee'].apply(classify_entrees_year)

df

,raw_title,year,month,day,director,date,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[""kad merad"", ""dany boon"", ""zoe felix""]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5,0.537844,190.3
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[""josiane balasko"", ""michel blanc"", ""marieanne...","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1,0.288597,188.8
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[""samy naceri"", ""frederic diefenthal"", ""emma w...","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1,0.447876,165.8
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[""hayden christensen"", ""natalie portman"", ""ewa...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29,0.564055,175.6
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[""donald glover"", ""beyonce"", ""seth rogen""]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21,0.348963,213.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""",2010-09-22,"[""jules pelissier"", ""ana girardot"", ""audrey ba...","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351,48033,40282,15,79945,93,6.4,-1,"[""francais""]",3,0.131239,207.1
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""",2003-12-10,"[""michael pitt"", ""louis garrel"", ""eva green""]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577,47922,-1,13,79849,-1,7.1,15000000,"[""anglais"", ""francais""]",2,0.167568,173.5
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[""toni servillo"", ""carlo verdone"", ""sabrina fe...","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654,47900,36844,13,198373,81,7.7,9200000,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60,0.181678,193.7
2668,Un baiser papillon,2011,6,1,"""karine silla""",2011-06-01,"[""valeria golino"", ""elsa zylberstein"", ""vincen...","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,-1,47715,38189,14,47715,170,5.4,3500000,"[""francais""]",2,0.133003,217.2


In [15]:
# f_acteurs = pd.DataFrame(df['casting'].str.strip('[]').str.split(',').explode().unique(), columns=['actor'])
# f_acteurs

In [16]:
df_actor = pd.read_sql_query('''SELECT actor, im.title, jp.first_week
 FROM films_imdb as im
 JOIN 
        films_jp jp ON im.id_jp = jp.id
  CROSS JOIN 
   JSON_TABLE(
     im.casting,
     "$[*]"
     COLUMNS(
       actor VARCHAR(255) PATH "$"
     )
   ) as aa
   order by actor''', engine)

In [17]:
df_actor = df_actor.groupby('actor')['first_week'].mean().reset_index() 

percentiles = df_actor['first_week'].sort_values(ascending=False).quantile([0.2, 0.4, 0.6, 0.8])
bins = [0] + list(percentiles) + [float('inf')]
labels = ['D', 'C', 'B', 'A', 'S']

df_actor['Category'] = pd.cut(df_actor['first_week'], bins=bins, labels=labels) 

import ast
df['casting'] = df['casting'].apply(ast.literal_eval)

def calculate_casting_avg(casting):
    casting_weeks = []
    for actor in casting:
        if actor in df_actor['actor'].values:
            actor_weeks = df_actor[df_actor['actor'] == actor]['first_week']
            casting_weeks.extend(actor_weeks)
    return sum(casting_weeks) / len(casting_weeks) if casting_weeks else 0

df['casting_avg_first_week'] = df['casting'].apply(calculate_casting_avg) 

percentiles = df['casting_avg_first_week'].sort_values(ascending=False).quantile([0.2, 0.4, 0.6, 0.8])

bins = [0] + list(percentiles) + [float('inf')]
labels = ['D', 'C', 'B', 'A', 'S']

df['casting_classement'] = pd.cut(df['casting_avg_first_week'], bins=bins, labels=labels, duplicates='drop')
df = df.drop('casting_avg_first_week', axis=1)
df

,raw_title,year,month,day,director,date,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,casting_classement
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[kad merad, dany boon, zoe felix]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5,0.537844,190.3,S
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[josiane balasko, michel blanc, marieanne chazel]","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1,0.288597,188.8,S
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[samy naceri, frederic diefenthal, emma wiklund]","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1,0.447876,165.8,S
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[hayden christensen, natalie portman, ewan mcg...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29,0.564055,175.6,S
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[donald glover, beyonce, seth rogen]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21,0.348963,213.2,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""",2010-09-22,"[jules pelissier, ana girardot, audrey bastien]","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351,48033,40282,15,79945,93,6.4,-1,"[""francais""]",3,0.131239,207.1,D
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""",2003-12-10,"[michael pitt, louis garrel, eva green]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577,47922,-1,13,79849,-1,7.1,15000000,"[""anglais"", ""francais""]",2,0.167568,173.5,C
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[toni servillo, carlo verdone, sabrina ferilli]","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654,47900,36844,13,198373,81,7.7,9200000,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60,0.181678,193.7,D
2668,Un baiser papillon,2011,6,1,"""karine silla""",2011-06-01,"[valeria golino, elsa zylberstein, vincent perez]","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,-1,47715,38189,14,47715,170,5.4,3500000,"[""francais""]",2,0.133003,217.2,D


In [18]:
# def classifier_acteurs(acteurs):
#     for acteur in acteurs:
#         if acteur in acteurs_tres_connus:
#             return 3
#         elif acteur in acteurs_fr:
#             return 2
#         elif acteur in acteurs_moins_connus:
#             return 1
#         else :
#             return 0

# df['classification_acteurs'] = df['casting'].apply(lambda x: classifier_acteurs(eval(x)))

# df


In [19]:
def classifier_pays(pays):
    if pays == 'etatsunis':
        return 3
    elif pays == 'france':
        return 2
    else :
        return 0

df['classification_country'] = df['country'].apply(lambda x: classifier_pays(x))

df

,raw_title,year,month,day,director,date,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,casting_classement,classification_country
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[kad merad, dany boon, zoe felix]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5,0.537844,190.3,S,2
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[josiane balasko, michel blanc, marieanne chazel]","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1,0.288597,188.8,S,2
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[samy naceri, frederic diefenthal, emma wiklund]","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1,0.447876,165.8,S,2
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[hayden christensen, natalie portman, ewan mcg...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29,0.564055,175.6,S,3
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[donald glover, beyonce, seth rogen]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21,0.348963,213.2,S,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""",2010-09-22,"[jules pelissier, ana girardot, audrey bastien]","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351,48033,40282,15,79945,93,6.4,-1,"[""francais""]",3,0.131239,207.1,D,2
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""",2003-12-10,"[michael pitt, louis garrel, eva green]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577,47922,-1,13,79849,-1,7.1,15000000,"[""anglais"", ""francais""]",2,0.167568,173.5,C,2
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[toni servillo, carlo verdone, sabrina ferilli]","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654,47900,36844,13,198373,81,7.7,9200000,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60,0.181678,193.7,D,0
2668,Un baiser papillon,2011,6,1,"""karine silla""",2011-06-01,"[valeria golino, elsa zylberstein, vincent perez]","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,-1,47715,38189,14,47715,170,5.4,3500000,"[""francais""]",2,0.133003,217.2,D,2


In [20]:
# Studios très connus
distributeurs_tres_connus = [
    'United International Pictures (UIP)',
    'Gaumont Buena Vista International (GBVI)',
    'Sony Pictures Releasing France',
    'Warner Bros. France',
    'The Walt Disney Company France',
    'Twentieth Century Fox France',
    'Columbia TriStar Films',
    '20th Century Studios',
    'Universal Pictures International France',
    'Pathé',
    'StudioCanal',
    'EuropaCorp Distribution',
    'Paramount Pictures France',
    'Universal Pictures France (UPF)'
]

# Studios moins connus
distributeurs_moins_connus = [
    'MK2 Diffusion',
    'Gaumont Distribution',
    'PolyGram Film Distribution',
    'Sony Pictures Home Entertainment',
    'Carlotta Films',
    'Bac Films',
    'Tamasa Distribution',
    'Mars Distribution',
    'Diaphana Films',
    'Wild Bunch Distribution',
    'Metropolitan FilmExport',
    'ARP Sélection',
    'SND',
    'Haut et Court',
    'Memento Distribution',
    'Le Pacte',
    'Ad Vitam',
    'Happiness Distribution',
    'Les Films du Losange',
    'Rezo Films',
    'Gebeka Films',
    'Les Films Number One',
    'Mary-X Distribution',
    'ARP / UGC',
    'Colifilms Diffusion',
    'Splendor Films',
    'AMLF',
    'UFD'
]




In [21]:
def calculate_distributor_scores(df):
    # 1. Fréquence de distribution
    frequency = df['distributor'].value_counts()

    # 2. Moyenne des spectateurs totaux
    avg_total_spectators = df.groupby('distributor')['total_spectator'].mean()

    # 3. Moyenne de la première semaine
    avg_first_week = df.groupby('distributor')['first_week'].mean()

    # 4. Moyenne du nombre de copies
    avg_copies = df.groupby('distributor')['copies'].mean()

    # Compilation des scores dans un DataFrame
    distributor_scores = pd.DataFrame({
        'frequency': frequency,
        'avg_total_spectators': avg_total_spectators,
        'avg_first_week': avg_first_week,
        'avg_copies': avg_copies
    })

    # Normalisation des scores
    max_values = distributor_scores.max()
    normalized_scores = distributor_scores / max_values

    # Combinaison des scores normalisés
    normalized_scores['distributor_combined_score'] = normalized_scores.mean(axis=1)

    return normalized_scores

# Appliquer la fonction pour calculer les scores
distributor_scores = calculate_distributor_scores(df)

# Joindre les scores combinés avec le DataFrame original
df = df.join(distributor_scores['distributor_combined_score'], on='distributor')
df

,raw_title,year,month,day,director,date,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,casting_classement,classification_country,distributor_combined_score
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[kad merad, dany boon, zoe felix]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5,0.537844,190.3,S,2,0.707143
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[josiane balasko, michel blanc, marieanne chazel]","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1,0.288597,188.8,S,2,0.593832
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[samy naceri, frederic diefenthal, emma wiklund]","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1,0.447876,165.8,S,2,0.541686
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[hayden christensen, natalie portman, ewan mcg...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29,0.564055,175.6,S,3,0.518926
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[donald glover, beyonce, seth rogen]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21,0.348963,213.2,S,3,0.489794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""",2010-09-22,"[jules pelissier, ana girardot, audrey bastien]","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351,48033,40282,15,79945,93,6.4,-1,"[""francais""]",3,0.131239,207.1,D,2,0.050279
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""",2003-12-10,"[michael pitt, louis garrel, eva green]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577,47922,-1,13,79849,-1,7.1,15000000,"[""anglais"", ""francais""]",2,0.167568,173.5,C,2,0.028472
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[toni servillo, carlo verdone, sabrina ferilli]","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654,47900,36844,13,198373,81,7.7,9200000,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60,0.181678,193.7,D,0,0.048923
2668,Un baiser papillon,2011,6,1,"""karine silla""",2011-06-01,"[valeria golino, elsa zylberstein, vincent perez]","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,-1,47715,38189,14,47715,170,5.4,3500000,"[""francais""]",2,0.133003,217.2,D,2,0.067727


In [22]:
def calculate_year_scores(df):
    # 1. Fréquence de distribution
    frequency = df['year'].value_counts()

    # 2. Moyenne des spectateurs totaux
    avg_total_spectators = df.groupby('year')['total_spectator'].mean()

    # 3. Moyenne de la première semaine
    avg_first_week = df.groupby('year')['first_week'].mean()

    # 4. Moyenne du nombre de copies
    avg_copies = df.groupby('year')['copies'].mean()

    # Compilation des scores dans un DataFrame
    distributor_scores = pd.DataFrame({
        'frequency': frequency,
        'avg_total_spectators': avg_total_spectators,
        'avg_first_week': avg_first_week,
        'avg_copies': avg_copies
    })

    # Normalisation des scores
    max_values = distributor_scores.max()
    normalized_scores = distributor_scores / max_values

    # Combinaison des scores normalisés
    normalized_scores['year_combined_score'] = normalized_scores.mean(axis=1)

    return normalized_scores

# Appliquer la fonction pour calculer les scores
year_scores = calculate_year_scores(df)

# Joindre les scores combinés avec le DataFrame original
df = df.join(year_scores['year_combined_score'], on='year')
df

,raw_title,year,month,day,director,date,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,casting_classement,classification_country,distributor_combined_score,year_combined_score
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[kad merad, dany boon, zoe felix]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5,0.537844,190.3,S,2,0.707143,0.830920
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[josiane balasko, michel blanc, marieanne chazel]","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1,0.288597,188.8,S,2,0.593832,0.855770
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[samy naceri, frederic diefenthal, emma wiklund]","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1,0.447876,165.8,S,2,0.541686,0.758796
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[hayden christensen, natalie portman, ewan mcg...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29,0.564055,175.6,S,3,0.518926,0.816782
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[donald glover, beyonce, seth rogen]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21,0.348963,213.2,S,3,0.489794,0.826926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""",2010-09-22,"[jules pelissier, ana girardot, audrey bastien]","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351,48033,40282,15,79945,93,6.4,-1,"[""francais""]",3,0.131239,207.1,D,2,0.050279,0.763881
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""",2003-12-10,"[michael pitt, louis garrel, eva green]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577,47922,-1,13,79849,-1,7.1,15000000,"[""anglais"", ""francais""]",2,0.167568,173.5,C,2,0.028472,0.710492
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[toni servillo, carlo verdone, sabrina ferilli]","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654,47900,36844,13,198373,81,7.7,9200000,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60,0.181678,193.7,D,0,0.048923,0.818377
2668,Un baiser papillon,2011,6,1,"""karine silla""",2011-06-01,"[valeria golino, elsa zylberstein, vincent perez]","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,-1,47715,38189,14,47715,170,5.4,3500000,"[""francais""]",2,0.133003,217.2,D,2,0.067727,0.881333


In [23]:
def calculate_country_scores(df):
    # 1. Fréquence de distribution
    frequency = df['country'].value_counts()

    # 2. Moyenne des spectateurs totaux
    avg_total_spectators = df.groupby('country')['total_spectator'].mean()

    # 3. Moyenne de la première semaine
    avg_first_week = df.groupby('country')['first_week'].mean()

    # 4. Moyenne du nombre de copies
    avg_copies = df.groupby('country')['copies'].mean()

    # Compilation des scores dans un DataFrame
    distributor_scores = pd.DataFrame({
        'frequency': frequency,
        'avg_total_spectators': avg_total_spectators,
        'avg_first_week': avg_first_week,
        'avg_copies': avg_copies
    })

    # Normalisation des scores
    max_values = distributor_scores.max()
    normalized_scores = distributor_scores / max_values

    # Combinaison des scores normalisés
    normalized_scores['country_combined_score'] = normalized_scores.mean(axis=1)

    return normalized_scores

# Appliquer la fonction pour calculer les scores
country_scores = calculate_country_scores(df)

# Joindre les scores combinés avec le DataFrame original
df = df.join(country_scores['country_combined_score'], on='country')


In [24]:
# def classifier_distributeurs(distributeur):
#     if distributeur in distributeurs_tres_connus:
#         return 1
#     elif distributeur in distributeurs_moins_connus:
#         return 2
#     else:
#         return 3

# df['distributor'] = df['distributor'].apply(classifier_distributeurs)

# df

In [25]:
# df["classification_acteurs"].nunique()

In [26]:
df.isnull().sum()

raw_title                     0
year                          0
month                         0
day                           0
director                      1
date                          0
casting                       0
distributor                   0
country                       0
duration                      0
genre                         0
first_day                     0
first_week                    0
first_weekend                 0
hebdo_rank                    0
total_spectator               0
copies                        0
rating_press                  0
budget                        0
lang                          0
award                         0
director_combined_score       1
entree_annee                  0
casting_classement            1
classification_country        0
distributor_combined_score    0
year_combined_score           0
country_combined_score        0
dtype: int64

In [27]:
colonnes_numeriques = ["first_day", 'entree_annee',"rating_press","first_weekend",'budget']

df[colonnes_numeriques] = df[colonnes_numeriques].replace(-1, np.nan)

imputer = KNNImputer(n_neighbors=7)

df_imputed = imputer.fit_transform(df[colonnes_numeriques])

df[colonnes_numeriques] = df_imputed

df.head()

,raw_title,year,month,day,director,date,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,casting_classement,classification_country,distributor_combined_score,year_combined_score,country_combined_score
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[kad merad, dany boon, zoe felix]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359.0,4378720,3586497.0,1,20489303,793,7.100000,11000000.0,"[""francais""]",5,0.537844,190.3,S,2,0.707143,0.830920,0.718586
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[josiane balasko, michel blanc, marieanne chazel]","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882.0,3906694,3235559.0,1,10355930,950,6.142857,35000000.0,"[""anglais"", ""italien"", ""francais""]",1,0.288597,188.8,S,2,0.593832,0.855770,0.718586
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[samy naceri, frederic diefenthal, emma wiklund]","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922.0,3478850,2951255.0,1,10345901,831,6.500000,70000000.0,"[""francais"", ""japonais"", ""allemand""]",1,0.447876,165.8,S,2,0.541686,0.758796,0.718586
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[hayden christensen, natalie portman, ewan mcg...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799.0,3303005,2878764.0,1,7247809,938,7.600000,113000000.0,"[""anglais""]",29,0.564055,175.6,S,3,0.518926,0.816782,0.917933
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[donald glover, beyonce, seth rogen]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478.0,3252896,2559370.0,1,10017995,680,6.800000,260000000.0,"[""anglais"", ""espagnol""]",21,0.348963,213.2,S,3,0.489794,0.826926,0.917933


In [28]:
df.isnull().sum()

raw_title                     0
year                          0
month                         0
day                           0
director                      1
date                          0
casting                       0
distributor                   0
country                       0
duration                      0
genre                         0
first_day                     0
first_week                    0
first_weekend                 0
hebdo_rank                    0
total_spectator               0
copies                        0
rating_press                  0
budget                        0
lang                          0
award                         0
director_combined_score       1
entree_annee                  0
casting_classement            1
classification_country        0
distributor_combined_score    0
year_combined_score           0
country_combined_score        0
dtype: int64

In [29]:
# df.nunique()

In [30]:
df['date'] = pd.to_datetime(df['date'])

df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

df.drop("date", axis=1, inplace=True)

In [31]:
def classify_season(month):
    if month == 12 or month == 1 or month == 2:
        return 'winter'
    elif month == 3 or month == 4 or month == 5:
        return 'spring'
    elif month == 6 or month == 7 or month == 8:
        return 'summer'
    elif month == 9 or month == 10 or month == 11:
        return 'autumn'


df['season'] = df['month'].apply(classify_season)


In [32]:
def classify_month_name(month):
    if month == 12:
        return 'december'
    elif month == 11:
        return 'novembre'
    elif month == 10:
        return 'october'
    elif month == 9:
        return 'september'
    elif month == 8:
        return 'august'
    elif month == 7:
        return 'july'
    elif month == 6:
        return 'june'
    elif month == 5:
        return 'may'
    elif month == 4:
        return 'april'
    elif month == 3:
        return 'march'
    elif month == 2:
        return 'february'
    elif month == 1:
        return 'january'



df['month_name'] = df['month'].apply(classify_month_name)

In [33]:
holiday_dates = SchoolHolidayDates()

df['is_holiday'] = df.apply(lambda row: holiday_dates.is_holiday_for_zone(datetime.date(row['year'], row['month'], row['day']), 'B'), axis=1)
df['is_holiday'] = df['is_holiday'].astype(int)
df

,raw_title,year,month,day,director,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,casting_classement,classification_country,distributor_combined_score,year_combined_score,country_combined_score,season,month_name,is_holiday
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""","[kad merad, dany boon, zoe felix]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359.0,4378720,3.586497e+06,1,20489303,793,7.100000,1.100000e+07,"[""francais""]",5,0.537844,190.3,S,2,0.707143,0.830920,0.718586,winter,february,0
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""","[josiane balasko, michel blanc, marieanne chazel]","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882.0,3906694,3.235559e+06,1,10355930,950,6.142857,3.500000e+07,"[""anglais"", ""italien"", ""francais""]",1,0.288597,188.8,S,2,0.593832,0.855770,0.718586,winter,february,0
2,Taxi 2,2000,3,29,"""gerard krawczyk""","[samy naceri, frederic diefenthal, emma wiklund]","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922.0,3478850,2.951255e+06,1,10345901,831,6.500000,7.000000e+07,"[""francais"", ""japonais"", ""allemand""]",1,0.447876,165.8,S,2,0.541686,0.758796,0.718586,spring,march,0
3,La Revanche des Sith,2005,5,18,"""george lucas""","[hayden christensen, natalie portman, ewan mcg...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799.0,3303005,2.878764e+06,1,7247809,938,7.600000,1.130000e+08,"[""anglais""]",29,0.564055,175.6,S,3,0.518926,0.816782,0.917933,spring,may,0
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""","[donald glover, beyonce, seth rogen]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478.0,3252896,2.559370e+06,1,10017995,680,6.800000,2.600000e+08,"[""anglais"", ""espagnol""]",21,0.348963,213.2,S,3,0.489794,0.826926,0.917933,summer,july,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""","[jules pelissier, ana girardot, audrey bastien]","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351.0,48033,4.028200e+04,15,79945,93,6.400000,3.442286e+07,"[""francais""]",3,0.131239,207.1,D,2,0.050279,0.763881,0.718586,autumn,september,0
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""","[michael pitt, louis garrel, eva green]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577.0,47922,1.122581e+05,13,79849,-1,7.100000,1.500000e+07,"[""anglais"", ""francais""]",2,0.167568,173.5,C,2,0.028472,0.710492,0.718586,winter,december,0
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""","[toni servillo, carlo verdone, sabrina ferilli]","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654.0,47900,3.684400e+04,13,198373,81,7.700000,9.200000e+06,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60,0.181678,193.7,D,0,0.048923,0.818377,0.340714,spring,may,0
2668,Un baiser papillon,2011,6,1,"""karine silla""","[valeria golino, elsa zylberstein, vincent perez]","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,19971.0,47715,3.818900e+04,14,47715,170,5.400000,3.500000e+06,"[""francais""]",2,0.133003,217.2,D,2,0.067727,0.881333,0.718586,summer,june,0


In [34]:
df.to_csv('Dataset_analyse.csv', index=False)
df

,raw_title,year,month,day,director,casting,distributor,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,casting_classement,classification_country,distributor_combined_score,year_combined_score,country_combined_score,season,month_name,is_holiday
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""","[kad merad, dany boon, zoe felix]","[""pathe renn productions"", ""hirsch"", ""les prod...",france,6360,comedie,558359.0,4378720,3.586497e+06,1,20489303,793,7.100000,1.100000e+07,"[""francais""]",5,0.537844,190.3,S,2,0.707143,0.830920,0.718586,winter,february,0
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""","[josiane balasko, michel blanc, marieanne chazel]","[""les films christian fechner"", ""tf1 films pro...",france,5820,comedie,537882.0,3906694,3.235559e+06,1,10355930,950,6.142857,3.500000e+07,"[""anglais"", ""italien"", ""francais""]",1,0.288597,188.8,S,2,0.593832,0.855770,0.718586,winter,february,0
2,Taxi 2,2000,3,29,"""gerard krawczyk""","[samy naceri, frederic diefenthal, emma wiklund]","[""arp selection"", ""canal"", ""leeloo productions""]",france,5400,comedie,801922.0,3478850,2.951255e+06,1,10345901,831,6.500000,7.000000e+07,"[""francais"", ""japonais"", ""allemand""]",1,0.447876,165.8,S,2,0.541686,0.758796,0.718586,spring,march,0
3,La Revanche des Sith,2005,5,18,"""george lucas""","[hayden christensen, natalie portman, ewan mcg...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",etatsunis,8760,fantasy,641799.0,3303005,2.878764e+06,1,7247809,938,7.600000,1.130000e+08,"[""anglais""]",29,0.564055,175.6,S,3,0.518926,0.816782,0.917933,spring,may,0
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""","[donald glover, beyonce, seth rogen]","[""walt disney pictures"", ""fairview entertainme...",etatsunis,7080,aventure action,630478.0,3252896,2.559370e+06,1,10017995,680,6.800000,2.600000e+08,"[""anglais"", ""espagnol""]",21,0.348963,213.2,S,3,0.489794,0.826926,0.917933,summer,july,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,Simon Werner a disparu,2010,9,22,"""fabrice gobert""","[jules pelissier, ana girardot, audrey bastien]","[""247 films"", ""canal"", ""cinecinema""]",france,5580,thriller,6351.0,48033,4.028200e+04,15,79945,93,6.400000,3.442286e+07,"[""francais""]",3,0.131239,207.1,D,2,0.050279,0.763881,0.718586,autumn,september,0
2666,Innocents The Dreamers,2003,12,10,"""bernardo bertolucci""","[michael pitt, louis garrel, eva green]","[""recorded picture company rpc"", ""fiction"", ""p...",france,6960,drame,7577.0,47922,1.122581e+05,13,79849,-1,7.100000,1.500000e+07,"[""anglais"", ""francais""]",2,0.167568,173.5,C,2,0.028472,0.710492,0.718586,winter,december,0
2667,La Grande Bellezza,2013,5,22,"""paolo sorrentino""","[toni servillo, carlo verdone, sabrina ferilli]","[""indigo film"", ""medusa film"", ""babe film""]",italie,8460,comedie dramatique,4654.0,47900,3.684400e+04,13,198373,81,7.700000,9.200000e+06,"[""italien"", ""japonais"", ""espagnol"", ""chinois""]",60,0.181678,193.7,D,0,0.048923,0.818377,0.340714,spring,may,0
2668,Un baiser papillon,2011,6,1,"""karine silla""","[valeria golino, elsa zylberstein, vincent perez]","[""europacorp"", ""grive productions"", ""france 2 ...",france,6060,drame,19971.0,47715,3.818900e+04,14,47715,170,5.400000,3.500000e+06,"[""francais""]",2,0.133003,217.2,D,2,0.067727,0.881333,0.718586,summer,june,0


In [35]:
df.isnull().sum()

raw_title                     0
year                          0
month                         0
day                           0
director                      1
casting                       0
distributor                   0
country                       0
duration                      0
genre                         0
first_day                     0
first_week                    0
first_weekend                 0
hebdo_rank                    0
total_spectator               0
copies                        0
rating_press                  0
budget                        0
lang                          0
award                         0
director_combined_score       1
entree_annee                  0
casting_classement            1
classification_country        0
distributor_combined_score    0
year_combined_score           0
country_combined_score        0
season                        0
month_name                    0
is_holiday                    0
dtype: int64